## Importing the Packages&Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import plotly

# Models to use
import lightgbm as lgb
#import catboost as cb

# Importing the metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix

# For measuring the training time taken during the fit process
import time

#from hyperopt import hp
#from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, space_eval

# Importing the Scalers
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import optuna

In [2]:
df = pd.read_csv('../input/higgs-cleaned/higgs_cleaned.csv')
df.head()

,class,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1pt,jet1eta,jet1phi,jet1b-tag,...,jet4eta,jet4phi,jet4b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
1,1,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
2,0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
3,1,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487
4,0,1.595839,-0.607811,0.007075,1.818450,-0.111906,0.847550,-0.566437,1.581239,2.173076,...,-0.654227,-1.274345,3.101961,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818


In [3]:
X, y = df.drop('class', axis=1), df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1864)

In [4]:
# df_train, df_test = train_test_split(df, test_size=0.2, random_state=1864)

## Normalizing the Features into range [0-1]

Scaling is surely an important part of the pipeline and I will be using MinMaxScaler to this end. One can turn the feature values into the standard normal range as well but the features do not always have the gaussian shape, I've done the basic tests for checking this in the baselines notebook. 

In [5]:
# There is no need to scale labels since they are already in the MinMaxScaler range [0-1]
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
#converting the dataset into proper LGB format 
d_train=lgb.Dataset(X_train, label=y_train)

For the meaning of search functions such as hp.uniform, you can take a look at here: http://hyperopt.github.io/hyperopt/getting-started/search_spaces/.

## Defining the Search Space & Objective Function

In [7]:
def objective(trial):
    
    classifier_parameters = {
    'learning_rate':    trial.suggest_float('learning_rate', 0.2, 0.3, step=0.005),
    'max_depth':        trial.suggest_int('max_depth', 6, 10, step=1),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 8, step=1),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.8, 1, step=0.05),
    'subsample':        trial.suggest_uniform('subsample', 0.7, 1),
    'num_iterations':     trial.suggest_categorical('num_iterations ', [150, 200, 350, 500]),
    'min_child_samples':trial.suggest_int('min_child_samples', 100, 300, step = 25),
    'num_leaves':       trial.suggest_int('num_leaves', 20, 50, step = 5),
    'objective':        'binary',
    'metric':           'auc',  
    'boosting_type':    'dart',
    'feature_pre_filter':False,
    'random_seed':      1864
}
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    cv_results = lgb.cv(classifier_parameters, d_train, nfold=5, verbose_eval = False, early_stopping_rounds=90, callbacks=[pruning_callback])
    best_auc = cv_results['auc-mean'][-1]
    #acc = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
    
    return best_auc

## Hyperparameter Tuning

We can now start the trials. Optuna is much easier to use compared to Hyperopt since the study object logs the important stuff regarding the trials already.

In [8]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction = 'maximize', study_name='Optuna_Basic_Study')
study.optimize(objective, n_trials=100, show_progress_bar=True)

/opt/conda/lib/python3.7/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:573: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 33247, number of negative: 29504
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017556 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[LightGBM] [Info] Number of data points in the train set: 62751, number of used features: 28
[LightGBM] [Info] Number of positive: 33246, number of negative: 29505
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020354 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[LightGBM] [Info] Number of data points in the train set: 62751, number of used features: 28
[LightGBM] [Info] Number of positive: 33246, number of negative: 29505
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017516 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[Lig

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/callback.py:223: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


...

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/callback.py:223: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.529808 -> initscore=0.119375
[LightGBM] [Info] Start training from score 0.119375
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.529808 -> initscore=0.119375
[LightGBM] [Info] Start training from score 0.119375
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.529816 -> initscore=0.119405
[LightGBM] [Info] Start training from score 0.119405


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:573: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 33247, number of negative: 29504
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019789 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[LightGBM] [Info] Number of data points in the train set: 62751, number of used features: 28
[LightGBM] [Info] Number of positive: 33246, number of negative: 29505
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[LightGBM] [Info] Number of data points in the train set: 62751, number of used features: 28
[LightGBM] [Info] Number of positive: 33246, number of negative: 29505
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017565 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[Lig

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


In [9]:
print('Best 5-Fold CV Score on train set:', round(study.best_value, 8))

Best 5-Fold CV Score on train set: 0.80829185


In [10]:
print('Best Parameters')
print('-'*50)
for k,v in study.best_params.items():
    print(k,':',v)

Best Parameters
--------------------------------------------------
learning_rate : 0.25
max_depth : 8
min_child_weight : 7
colsample_bytree : 0.9500000000000001
subsample : 0.9196633665123095
num_iterations  : 350
min_child_samples : 175
num_leaves : 50


In [11]:
print('Best Trial Index:', study.best_trial.number)

Best Trial Index: 27


## Plots of the Initial Hyperparameter Search

In [12]:
#import plotly.offline as pyo
#pyo.init_notebook_mode()

In [13]:
optuna.visualization.plot_param_importances(study)

In [14]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

In [15]:
optuna.visualization.plot_slice(study)

In [16]:
optuna.visualization.plot_optimization_history(study)

In [17]:
optuna.visualization.plot_parallel_coordinate(study)

## Applying the Tuned Model on Test Data

In [18]:
best_params = study.best_params

clf = lgb.LGBMClassifier(**best_params)
clf.fit(X_train, y_train, verbose=False)

preds = clf.predict(X_test)

print(accuracy_score(y_test, preds))

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning:

'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.



[LightGBM] [Warning] num_iterations is set=, num_iterations=100 will be ignored. Current value: num_iterations=
[LightGBM] [Warning] Unknown parameter: 350
0.7231514533401325
